In [ ]:
import json
import openai
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from together import Together
import re

def query_openai(client, prompt, model, temperature, max_tokens, mode):

    try:
        if mode == "o1":
            client = openai
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. Limit your response to 500 tokens."},
                    {"role": "user", "content": prompt},
                ])
            return response
        elif mode == "together":
            os.environ['TOGETHER_API_KEY'] = ''
            client = Together()
            response = client.chat.completions.create(
                model="deepseek-ai/DeepSeek-R1",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. Limit your response to 130 tokens."},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                top_p=1,
                seed=42
            )
 
            return response
        else:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                stream=False
            )
            return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def determine_vulnerability(response):
    prompt = f"Determine if the following response indicates a vulnerability was found:\n\n{response}\n\nRespond ONLY with 'vulnerable' or 'non-vulnerable'. Answer in this format: ['response': 'vulnerable'] or  ['response': 'non-vulnerable']"
    completion = query_openai(
        client=openai,
        prompt=prompt,
        model="deepseek-ai/DeepSeek-R1",
        temperature=0.6,
        max_tokens=4096,
        mode = "together"
    )
 
 
    pattern = r"n<\/think>\n\n\['response': '(vulnerable|non-vulnerable)'\]"
    print("-------------")
    print(response)
    print("-------------")

    if completion is not None:
 
        content = completion.choices[0].message.content.strip().lower()
        print("******************")
        print(content)
        print("******************")
 
        if "['response': 'vulnerable']" in content:
            result = "vulnerable"
        elif "['response': 'non-vulnerable']" in content:
            result = "non-vulnerable"
        else:
            result = "inconclusive"


    else:
 
        return "ERROR"

def process_json(input_file, output_file, max_workers=10):

    with open(input_file, 'r') as f:
        data = json.load(f)

    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_key = {executor.submit(determine_vulnerability, response): key for key, response in data.items()}

        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                results[key] = future.result()
            except Exception as e:
                print(f"Error processing {key}: {e}")
                results[key] = "ERROR"

 
 


In [ ]:
 
input_file = "/Users/ic/Desktop/LLM-as-Static-Proxy-Test/experiments_latest/cwe-476/exp_17/func_src_before/output_responses_deepseek-ai_DeepSeek-R1_temp0.6_20250212183618_trial_5.json"
output_file = None

 
process_json(input_file, output_file, max_workers=5)


-------------
<think>
Alright, let's tackle this code example. I need to check each of the six tests to see if there are any vulnerabilities.

Starting with Test 1: Check if pointers are checked for null before dereferencing. Looking through the code, there are several pointers. For instance, 'skb' is allocated with alloc_skb, and there's a check if (!skb), which is good. Then, after skb is assigned, they proceed to set headers. But wait, later on, there's a line like ip_hdr(skb)->protocol = IPPROTO_UDP;. Wait, 'skb' was checked after allocation, but what if alloc_skb returns NULL? They do check if (!skb) and handle the error by setting err and jumping to errout. So in that case, they avoid dereferencing a null skb. But what about other pointers?

Looking at 'rt', which is a pointer to an rtable. In the else block, there's 'rt = ip_route_output_key_hash_rcu(...);' followed by a check if IS_ERR(rt). Then, they set skb_dst_set(skb, &rt->dst). But if rt is an error pointer, they set err. 